# Q1 Sorting with duplicates (30 marks)

The input is a unsorted list of non-negative real numbers. The list has size $n$, and there are $k \leq n$ different numbers in that list (i.e. there may be duplicates).

## Q1.1 (25 marks)
Design and implement an algorithm which can sort this list and has *average* runtime in $O(n \log k)$. You may only use the Python $list$ container, and no other (so no dictionary, no set, etc). You may not use an existing implementation of a sorting algorithm (including Python's). If you feel that it is needed, you can write your own implementation of a sorting algorithm. Any other data structure that you need has to be implemented here as well. For reference, the solution code is less than 40 lines long.

In [0]:
def sortlist(l):
    """ Returns a sorted list which contains the same elements as l
        (including duplicates). The input list l should not be changed."""
    hashList = [None]*len(l)
    hlLen = len(l)
    # Iterating through the list. (n times)
    for v in l:
        # If the slot is empty
        if hashList[int(v)%hlLen] == None:
            hashList[int(v)%hlLen] = [v, 1]
        # If the slot already contained the value
        elif hashList[int(v)%hlLen][0] == v:
            hashList[int(v)%hlLen][1] += 1
        # Slot has been occupied, the value inside does not match
        # Search for another available slot.
        else:
            # move the index forward by one.
            index = (int(v)+1)%hlLen
            # found flag
            found = False
            # index offset factor
            offset = 1
            # Searching
            while hashList[index] != None:
                # The value is stored in another slot
                if hashList[index][0] == v:
                    hashList[index][1] += 1
                    found = True
                    break
                # move the index forward by offset
                index = (index + offset)%hlLen
                # offset increment
                #offset += 1
            # While loop exit when no target value inside and an empty slot found.
            if not found:
                hashList[index] = [v, 1]
    # Remove empty slot.
    uniqueList = []
    # Iterating through the hash list. (n times)
    for i in range(hlLen):
        # Visiting a not none slot, put the payload into unique list.
        if hashList[i] != None:
            uniqueList.append(hashList[i])
    # Sorting.
    return quicksort(uniqueList)
def quicksort(l):
    # Base case where the sublist length is 1.
    if len(l) == 1:
        return [l[0][0]]*l[0][1]
    # Base case where the sublist is empty
    elif len(l) == 0:
        return []
    # Pick the first element as pivot.
    pivot1 = l[0]
    # Split the sublist into two sub-sublists, 
    # where item smaller than the pivot is put into the left one, vice versa.
    leftList = []
    rightList = []
    for i in range(1, len(l)):
        # Smaller than pivot
        if l[i][0] < pivot1[0]:
            leftList.append(l[i])
        # Larger than or equal to.
        else:
            rightList.append(l[i])
    # Recursive sort the left list. The returned result should be already sorted.
    leftSorted = quicksort(leftList)
    # Resursive sort the right list. 
    rightSorted = quicksort(rightList)
    # Concatenate the left list and the pivot.
    leftSorted.extend([pivot1[0]]*pivot1[1])
    # Concatenate what was concatenated previously.
    leftSorted.extend(rightSorted)
    # Now in the leftSorted, the sublist are fully sorted.
    return leftSorted

We provide unit tests below. Your implementation must pass all tests to get full marks. For reference, the solution code runs in less than a minute. If your code requires a significant longer time to run, then it may be bugged, or your algorithm may not have an average runtime in $O(n \log k)$.

In [0]:
import unittest
import random
import math
random.seed(a=0)

class Testsortlist(unittest.TestCase):
    def setUp(self):
        self.numbers = [math.sqrt(i) for i in random.sample(range(10000000), 1000)]
        self.ntests = 100
    
    def checklist(self, l):
        check = sortlist(l)
        l.sort()
        self.assertEqual(l, check)
        
    def testEmpty(self):
        self.checklist([])
        
    def testSingle(self):
        self.checklist([0])
        
    def testNoMult(self):
        for _ in range(self.ntests):
            self.checklist(random.sample(self.numbers, len(self.numbers)//2))
            
    def testkMult(self):
        for k in [2, 5, 10, 100]:
            for _ in range(self.ntests):
                self.checklist(random.choices(self.numbers, k=k*len(self.numbers)))

In [0]:
testlist = Testsortlist()
suite = unittest.TestLoader().loadTestsFromModule(testlist)
unittest.TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 65.204s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

## Q1.2 (5 marks)
Prove that your algorithm has an average runtime in $O(n \log k)$. Determine and prove its worst-case runtime.

The sorting procedure is divided into two steps.

### 1 Constructing <number, times> table.

This step takes advantage of **HashList** where visiting an item cost constant time averagely. Inside each cell, it's a list of 2 items representing number and the number of times it appears in the list. 

Once all the items are counted, another space of list is allocated to place the statistic result. Then, it is fed into the next stage.

In this step, it will take $n$ times to iterate the whole list. In each iteration, computing hash value is in $O(1)$. Assignment costs $O(1)$ in average. But it is possible to cost $O(n)$ while collision encountered. The worst case is that collision is happened through out the whole process. Therefore, it is $O(n^2)$ for wost case.

### 2 Re-constructing the sorted list by quicksort.

This steps accepts the list with unique items and their number of times to construct a sorted list. To be detailed, each recursive call takes the first item as the pivot, then comparing the subsequent items and putting them into two lists, one of which contains all the items smaller than pivot, vice versa for the second list. 

Recursive call terminates when reaching the base case in which the length of input list is 0 or 1. When the input list has only one item, reconstructing the list based on the number of times and return this list. While back to the caller, it will receive two sorted list. Merging process is simply connecting two lists plus the reconstructed pivot.

Assuming that the length of two sublists are roughly equal in average case. 

$T(n) = 2T(\frac{k}{2}) + cn$ (extend of list costs about $\frac{n}{2}$ time, so it is in $O(n)$, $c$ is constant) 

$T(n) = 2(2T(\frac{k}{4}) + cn) + cn$

$T(n) = 2(2(2T(\frac{k}{8})) + cn) + cn$

$\dots$

When reached the base case, the function has been called $\log k$ times. 

$T(n) = (1 + 2 + 4 + \dots + 2^{\log k})cn = (1 + 2 + 4 + \dots + k)cn$

So, there are $\log k$ terms to sum, from $1$ to $k$.

$T(n) \in O(n\log k)$